In [1]:
import os, sys
import django
sys.path.insert(0, '../')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'ratemympa.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from django.contrib.gis.gdal import DataSource
from django.contrib.gis import geos
from django.contrib.gis.geos.prototypes.io import wkt_w

In [3]:
ds = DataSource('../../ratemympa/downloads/south_africa/MPA_UPDATE.shp')
lyr = ds[0]

In [4]:
lyr.num_feat
stage_lookup = {
    'Implement': 'implemented',
    'Designated': 'designated',
    'Actively M': 'actively managed',
}
level_lookup = {
    'Fully': 'full',
    'Highly': 'high',
    'Lightly': 'light',
    'Minimally': 'minimal',
}

In [5]:
# Create a dictionary to hold the south africa features for each wdpa_id
sites = {}

for feat in lyr:
    # Iterate over the features and add them to the dictionary
    wdpa_id = int(feat.get('WDPAID'))
    if wdpa_id not in sites:
        sites[wdpa_id] = []
    geom = feat.geom.geos.transform('4326', clone=True)
    # Strip Z-dimension
    if geom.hasz:
        wkt2d = wkt_w(dim=2).write(geom).decode()
        geom = geos.GEOSGeometry(wkt2d, srid=4326)
    if isinstance(geom, geos.Polygon):
        geom = geos.MultiPolygon(geom)
    zone = {
        'geom': geom,
        'site_name': feat.get('CUR_NME').replace(' Marine Protected Area', ''),
        'site_designation': 'Marine Protected Area',
        'zone_name': feat.get('CUR_ZON_NM'),
        'zone_type': feat.get('CUR_ZON_TY'),
        'zone_code': feat.get('CUR_ZON_CD'),
        'region': feat.get('MPA_REGION'),
        'desig_date': feat.get('D_DCLAR'),
        'designated': True,
        'area_marine': feat.get('GIS_AREA'),
        'area_total': feat.get('GIS_AREA_1'),
        'stage': stage_lookup[feat.get('Stage')],
        'level': level_lookup[feat.get('Level')],
        'region': feat.get('MPA_REGION'),
    }
    sites[wdpa_id].append(zone)

In [6]:
from mpa.models import Site, Mpa
# Iterate over the dictionary and create Site and Zone objects
for wdpa_id, zones in sites.items():
    msite = Site.objects.filter(wdpa_id=wdpa_id).first()
    print(wdpa_id, zones[0]['site_name'])
    if msite:
        print('---', msite.site_id, msite.name)
    else:
        msite = Site()
    msite.wdpa_id = wdpa_id
    msite.name = zones[0]['site_name']
    msite.designation = zones[0]['site_designation']
    # msite.sovereign = 'ZAF'
    # msite.country = 'ZAF' 
    # msite.designated_date = zones[0]['desig_date'].strftime('%Y-%m-%d')
    # msite.establishment_stage = zones[0]['stage']
    msite.save()

    # Create ZoneAssessment objects and add them to the SiteAssessment
    for zone in zones:
        mzone = Mpa.objects.create(
            geom = zone['geom'],
            site = msite,
            wdpa_id = wdpa_id,
            name = zone['zone_name'], 
            designation = zone['site_designation'], 
            sovereign = 'ZAF',
            country = 'ZAF',
            status_year = int(zone['desig_date'].strftime('%Y')),
            implemented = zone['stage'] in ('implemented', 'actively managed'),
            implementation_date = zone['desig_date'] if zone['stage'] in ('implemented', 'actively managed') else None,
            calc_area = zone['area_total'],
            calc_m_area = zone['area_marine'],
            other_ids = zone['zone_code'],
        )

315479 De Hoop
300546 Dwesa-Cwebe
555571018 Agulhas Bank Complex
555571029 Addo Elephant National Park
555571023 Cape Canyon
555571022 Benguela Muds
555571019 Agulhas Muds
555571026 Namaqua Fossil Forest
555571011 Port Elizabeth Corals
555571014 Childs Bank
306179 Aliwal Shoal
--- 76 Aliwal Shoal
555571020 Robben Island
555571032 Amathole Offshore
--- 77 Amathole Offshore
555571028 Protea Banks
--- 78 Protea Banks
555571025 Southeast Atlantic Seamounts
555571016 iSimangaliso
555571013 Browns Bank Corals
555571031 uThukela
--- 79 uThukela
306178 Table Mountain National Park
306181 Pondoland
555563466 Stilbaai
555563467 Walker Bay Whale Sanctuary
315492 Tsitsikamma
555571021 Orange Shelf Edge
555571027 Namaqua
555512160 Prince Edward Island
--- 5 Prince Edward Islands
300543 Hluleka
315477 Betty's Bay
315481 Helderberg
315482 Jutten Island
315484 Malgas Island
315486 Marcus Island
315487 Robberg
315488 Sardinia Bay
315489 Sixteen Mile Beach
315491 Trafalgar
315483 Langebaan Lagoon
555571